In [ ]:
# Removed Multithread Processing from Read Directory. Need to have displine and Principles related to 
# Creation of Functions, Create Individual functions which do components, opposed to long ones. 

In [2]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [ ]:
df['Learning Type'].value_counts()

In [8]:
df.columns

Index(['Process', 'Categorization', 'Word', 'Definition', 'Notes', 'Link',
       'Image', 'Markdown Equation', 'Dataset Size', 'Learning Type',
       'Algorithm Class', 'Model', 'Ensemble'],
      dtype='object')

In [10]:
df['Model'].value_counts()

Model
Linear        4
Tree          1
Meta-Model    1
Name: count, dtype: int64

In [9]:
df['Algorithm Class'].value_counts()

Algorithm Class
Classification              4
Regression                  2
Density-based clustering    1
Name: count, dtype: int64

In [6]:
df['Learning Type'].value_counts()

Learning Type
Supervised    6
Name: count, dtype: int64

In [4]:
df['Categorization'].value_counts()

Categorization
Algorithm                    201
Procedure                     22
Operational Definition        18
Pipeline Stage                13
Definition                    13
Optimizer                      7
Semantic Type                  6
Function                       4
Model Type                     4
Concept                        4
Feature Selection              4
Centroid-based clustering      3
Regularization                 3
General Principles             3
Functional Role                3
Clustering                     3
Theorm                         2
Model Architecture             2
Density-based clustering       2
Theorem                        1
Property                       1
Evaluation                     1
Probabilistic clustering       1
Graph-based clustering         1
Insance Based Learning         1
Name: count, dtype: int64

In [3]:
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

df = pd.read_csv(google_definition_csv)
df



,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Data Preparation,Operational Definition,Baseline,Baseline represents the typical or expected be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Preparation,Operational Definition,Dimension,A Dimension is a conceptual axis of behavior o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data Preparation,Operational Definition,Implementation,An Implementation is a specific measurable rep...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data Preparation,Operational Definition,Momentum,Momentum represents the direction and rate of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data Preparation,Operational Definition,Parameter,A Parameter is a tuning choice that controls h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Organization,Definition,Procedure,"Is a detailed, step by step set of instruction...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319,Organization,Definition,RASCI,RASCI is a responsibility assignment framework...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,Organization,Definition,Ownership,State of having end-to-end accountability for ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,Organization,Definition,Stewardship,"Responsibility to safeguard, maintain, and opt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
from data_d_strings import template_doc_string_print

template_doc_string_print()



    Definition of Function

    Parameters:
        List of Parameters

    Returns:
        Object Type

    date_created:30-Dec-25
    date_last_modified: 30-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        Example Function Call



In [2]:
from shared_folder import read_directory
d_py_function =  '/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/'
read_directory(d_py_function)

['utility_functions.py',
 '.DS_Store',
 'dict_processing.py',
 'Archive',
 'data_d_lists.py',
 'shared_folder.py',
 'list_processing.py',
 'connections.py',
 '__init__.py',
 '__pycache__',
 'string_processing.py',
 'V2',
 'input_functions_ignore.py',
 'sql_.py',
 '.ipynb_checkpoints',
 'data_d_strings.py',
 'data_d_dicts.py',
 'df_processing.py']

In [3]:


# Import a .TXT or .PY File
location = '/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/shared_folder.py'

from shared_folder import text_file_import
file = text_file_import(location)

from shared_folder import parse_dot_py_file
function_list, function_parameters = parse_dot_py_file(file)

## Generate a Summary File for a Folder.
from shared_folder import parse_dot_py_folder
function_list, function_parameters = parse_dot_py_folder()

from shared_folder import create_py_table_dict
create_py_table_dict()

In [6]:
import pandas as pd
import webbrowser
import datetime
import requests
import os

DownloadFilesFromGit()

BackUpGoogleSheets()

True

In [ ]:


import pandas as pd
import numpy as np

def TranposeNonTimeSeriesDF(df, index, columns=None):
    '''
    Transposes a non-time-series DataFrame from wide to long format by melting specified columns.

    This is especially useful for flattening columns into a single column to support tools 
    like Power BI, where long format enables dynamic pivoting and aggregation.

    Parameters:
        df (DataFrame): The input pandas DataFrame.
        index (list): Columns to retain as identifiers (will remain unchanged).
        columns (list): Columns to unpivot into key-value pairs.

    Returns:
        DataFrame: A long-format DataFrame with 'variable' and 'value' columns.
    '''
    if not columns:
        columns = [col for col in final_df1.columns if (isinstance(col, pd.Timestamp))|(isinstance(col, datetime.datetime))]
    
    return df.melt(id_vars=index, value_vars=columns)

def CreatePivotTableFromTimeSeries(df,
                                   index,
                                   columns,
                                   values,
                                   aggfunc='sum',
                                   skipna=True):
    
    '''
    Function to Summaryize a Time Series Dataframe into a Pivot. Creating a number of critical Metrics.
    
    
    
    '''
    
    # 1. Pivot
    if index==None:
        df1 = df.pivot_table(columns=columns,values=values,aggfunc=aggfunc)
    else:
        df1 = df.pivot_table(index=index, columns=columns, values=values, aggfunc=aggfunc)

    # 2. Capture original month columns IMMEDIATELY after pivot
    month_cols = df1.columns.tolist()
 
    # 3. Add rolling window stats
    if len(month_cols) >= 3:
        df1['AVG_3M'] = df1[month_cols[-3:]].mean(axis=1, skipna=skipna)
        df1['CHG_3M'] = df1[month_cols[-1]]-df1[month_cols[-3]]
        try:
            df1['PERC_CHG_3M'] = df1['CHG_3M']/df1[month_cols[-3]]
        except:
            df1['PERC_CHG_3M'] = 0
    
    if len(month_cols) >= 6:
        df1['AVG_6M'] = df1[month_cols[-6:]].mean(axis=1, skipna=skipna)
        df1['CHG_6M'] = df1[month_cols[-1]]-df1[month_cols[-6]]
        try:
            df1['PERC_CHG_6M'] = df1['CHG_6M']/df1[month_cols[-6]]
        except:
            df1['PERC_CHG_6M'] = 0
            
    if len(month_cols) >= 12:
        df1['AVG_12M'] = df1[month_cols[-12:]].mean(axis=1, skipna=skipna)
        df1['CHG_12M'] = df1[month_cols[-1]]-df1[month_cols[-12]]
        try:
            df1['PERC_CHG_12M'] = df1['CHG_12M']/df1[month_cols[-12]]
        except:
            df1['PERC_CHG_12M'] = 0

    df1['CHG_DF']  = df1[month_cols[-1]]-df1[month_cols[0]]
    df1['AVG_DF'] = df1[month_cols[-1:]].mean(axis=1, skipna=skipna)
    df1['PERC_CHG_DF'] = df1['AVG_DF']/df1[month_cols[-1]]

    
    # 4. Now calculate global stats **only using the original month columns**
    stats = pd.DataFrame({
        'MEAN': df1[month_cols].mean(axis=1, skipna=skipna),
        'STD': df1[month_cols].std(axis=1, skipna=skipna),
        'MAX': df1[month_cols].max(axis=1, skipna=skipna),
        'MIN': df1[month_cols].min(axis=1, skipna=skipna),
        'COUNT': df1[month_cols].count(axis=1)
    })

    # 5. Merge the stats
    df1 = pd.concat([df1, stats], axis=1)
    
    return df1.fillna(0)


def CreateMultiplePivotTableFromTimeSeries(df,
                                           index_list,
                                           metric_list,
                                           column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    

    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    
    for metric in metric_list:
        print(f'Attempting to Process:{metric}')
        try:
            all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
            cols = list(all_df.columns)
            all_df = all_df.reset_index(drop=True)
            all_df['METRIC'] = metric
            cols.insert(0,'METRIC')

            for key in index_list:
                cols.insert(0,key)
                all_df[key] = 'All'

            final_df = pd.concat([final_df,all_df[cols]])
            # Iterate through all Index Items Individually
            for key in index_list:
                temp = CreatePivotTableFromTimeSeries(df,
                                                      index=key,
                                                      values=metric,
                                                      columns=column).reset_index() 
                for missing in [x for x in index_list if x != key]:
                    temp[missing] = 'All'
                temp['METRIC'] = metric
                final_df = pd.concat([final_df,temp])

            # Add Value for Metric with Entire Index Combination
            temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        except:
            print(f'Could Not Process Metric:{metric}.')

    return final_df




def CreateMultiplePivotTableFromTimeSeries(df,index_list,metric_list,column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    
    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    for metric in metric_list:
        all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
        cols = list(all_df.columns)
        all_df = all_df.reset_index(drop=True)
        all_df['METRIC'] = metric
        cols.insert(0,'METRIC')

        for key in index:
            cols.insert(0,key)
            all_df[key] = 'All'

        final_df = pd.concat([final_df,all_df[cols]])

        # Iterate through all Index Items Individually
        for key in index_list:
            temp = CreatePivotTableFromTimeSeries(df,index=key,
                                                  values=metric,
                                                  columns=column).reset_index() 
            for missing in [x for x in index if x != key]:
                temp[missing] = 'All'
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        
        # Add Value for Metric with Entire Index Combination
        temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
        temp['METRIC'] = metric
        final_df = pd.concat([final_df,temp])
    
    return final_df


def SummarizeTimeSeriesDf(df,
                          summary_cols,
                          primary_key_list):
    '''
    Function to Summarize a Time Series dataframe based on a finite number of identified Columns.
    
    Parameters
        df (Dataframe): TimeSeries in Nature
        summary_cols (List): List of Columns which are to be included in SUmmary
        primary_key_list (list): Primary Key of Dataframe
    
    Returns
        temp_df1: Raw Data of SUmmary Cols with a Count of Observations. If include Month Variable Easy to add to Pivot Table
        summary: Summary (Excluding Primary Key). including Total Observations, MEan, Max, Min.
    
    
    '''
    temp_df = df[summary_cols].copy()
    temp_df['COUNT'] = 1
    
    # Unique Occurances by Pivot Criteria. Important to Include Month
    temp_df1 = temp_df.groupby(summary_cols).sum().reset_index().rename(columns={'COUNT':'TOTAL_DAYS'})
    
    pivot_columns1 = [x for x in summary_cols if x not in primary_key_list]
    
    summary = temp_df1.groupby(pivot_columns1).agg(
        TOTAL=('TOTAL_DAYS', 'count'),
        AVG_DAYS_OPEN=('TOTAL_DAYS', 'mean'),
        MAX_OBS=('TOTAL_DAYS', 'min'),
        MIN_OBS=('TOTAL_DAYS', 'max')).reset_index()
    
    return temp_df1,summary


import pandas as pd
import numpy as np
from scipy.stats import norm

def SampleDataFrame(df, 
                    conf=.95, 
                    me=0.05,
                    mv=0.5,
                    print_=0,
                    new_column_name=""):
    """
    Returns a random sample from a DataFrame based on confidence level and margin of error.

    Parameters:
        df (pd.DataFrame): The dataset to sample from.
        conf(float): Desired Confidence Percentage Level (e.g., 90, 95, 99).
        me (float): Margin of Error, (default is 5%).
        mv (float): Maximum Variability (Expected Level of Default)

    Returns:
        pd.DataFrame: A random sample of the required size.
    """
    df = df.copy()
    
    if not 0 <= mv <= 1:
        raise ValueError("mv (failure rate) must be between 0 and 1.")

    N = len(df)
    if N == 0:
        raise ValueError("DataFrame is empty")

    # Calculate the Z-score based on the confidence level
    z = norm.ppf(1 - (1 - conf) / 2)
    

    # Calculate the initial sample size (without finite population correction)
    n0 = (z**2 * mv * (1 - mv)) / (me**2)
    
    # Apply finite population correction if the population is smaller than 100,000
    if N >= 10000:  # For large populations, skip the correction
        n = int(n0)
    else:
        n = int((n0 * N) / (n0 + N - 1))

    if print_==1:
        print(f"Z-score: {z}")  # Debug Z-score
        print(f"Initial sample size (n0): {n0}")  # Debug n0
        print(f"Sample size with FPC: {n}")  # Debug final sample size
    
    sample = df.sample(n=n, random_state=42)
    
    if len(new_column_name)==0:
        return sample 

    else:
        sample_index = sample.index
        df[new_column_name] = 0
        df.loc[sample_index, new_column_name] = 1
        return df

def ReviewEntireDataframe(df,file_name=None):
    
    final_df = pd.DataFrame()
    
    for column in df.columns:
        start_time = timeit.default_timer()
        temp_df = ColumnStatisticalReview(df,column)
        print(f'Elapsed time to process {column}:{timeit.default_timer() - start_time:,.2f}')
        final_df = pd.concat([final_df,temp_df],axis=1)
    if file_name:
        final_df.to_csv(f"{file_name}.csv")
        
    return final_df

def ColumnStatisticalReview(df,
                            column_name,
                            partitions=10,
                            top_x_records=10,
                            exclude_blanks_from_segments=1,
                            exclude_zeroes_from_segments=1):

    '''
    Function to Conduct a Simple Statistical Review of a Column, Including Understanding the positional distribution
    of values. 

    Args:
        column_name (str): Name of Column

        partitions (int): Number of partitions to include (Decile 10)

        exclude_blanks_from_segments (int): Binary Flag, whether to exclude Blank Values from Segment determination.
        If blank values are excluded it gives a better representation for the members of the set, however it might 
        provide a misleading representation of the population.

        exclude_zeroes_from_segments (int): As above, with respect to 0 values. Is processed after exclude_blanks, as
        such it can include both blanks and true 0 values. 


    '''

    temp_dict = {}
    
    is_numeric = pd.api.types.is_numeric_dtype(df[column_name])
    
    if is_numeric:
        temp_dict['SUM'] = df[column_name].sum()
        temp_dict['MEAN'] = df[column_name].mean()
        temp_dict['STD_DEV'] =  df[column_name].std()
        temp_dict['MEDIAN'] = df[column_name].median()
        temp_dict['MAX'] = df[column_name].max()
        temp_dict['MIN'] = df[column_name].min()
        
    temp_dict['TOTAL_RECORDS'] = len(df)
    temp_dict['UNIQUE_RECORDS'] = len(df.drop_duplicates(column_name))
    temp_dict['NA_RECORDS'] = len(df[df[column_name].isna()])
    temp_dict['NULL_RECORDS'] = len(df[df[column_name].isnull()])
    
    if is_numeric:
        temp_dict['ZERO_RECORDS'] = len(df[df[column_name]==0])
        temp_dict['NON_ZERO_RECORDS'] = len(df[df[column_name]!=0])    

    temp_df = pd.DataFrame(temp_dict.values(),index=temp_dict.keys(),columns=[column_name])
    
    if temp_dict['TOTAL_RECORDS']==len(df[df[column_name].isnull()]):
        return temp_df

    # Add top X records Based on Frequency
    if top_x_records>0:
        top_instances = pd.DataFrame(df[column_name].value_counts(dropna=False).head(top_x_records)).reset_index().rename(columns={column_name:'count','index':column_name})
        if len(top_instances)>0:
            top_instances[column_name] = top_instances.apply(lambda row: f"Value: {row[column_name]}, Frequency: {row['count']}", axis=1)
            top_instances['index'] = [f"Top {x+1}" for x in range(len(top_instances[column_name]))]
            top_instances = top_instances.drop('count',axis=1).set_index('index')
            temp_df = pd.concat([temp_df,top_instances])
        
    if (partitions>0)&(pd.api.types.is_numeric_dtype(df[column_name]))&(temp_dict['UNIQUE_RECORDS']>1):
        segment_df = ColumnPartitioner(df=df,
                                       column_name=column_name,
                                       partitions=partitions,
                                       exclude_blanks=exclude_blanks_from_segments,
                                       exclude_zeros=exclude_zeroes_from_segments,
                                       return_value='')
        
        seg_val_df = ColumnPartitioner(df=df,
                                           column_name=column_name,
                                           partitions=partitions,
                                           exclude_blanks=exclude_blanks_from_segments,
                                           exclude_zeros=exclude_zeroes_from_segments,
                                           return_value='agg_value').rename(columns={'VALUE':column_name})

        return pd.concat([temp_df,segment_df.T,seg_val_df])
    return temp_df

def CompareFunction(func1,func2,additional_records=20):
    
    '''
    Function which Compares 2 Functions and determines if they are different. Specifically, it can help to easily
    Manage Version control of Functions outside of a More robust environment such as GIT.
    

    '''
    
    list1 = FunctionToSTR(func1)
    list2 = FunctionToSTR(func2)
    
    length = max(len(list1),len(list2))
    
    for record in range(0,length):
        if list1[record]==list2[record]:
            if record == (length-1):
                print("All Records Reconcile")
            pass
        else:
            try:
                print(list1[record:record+additional_records])
                print(list2[record:record+additional_records])
            except:
                print(list1[record:record:])
                print(list2[record:record:])
            break


In [ ]:
import pandas as pd
import webbrowser
import datetime
import requests
import os


def DownloadFilesFromGit(user='derek-dewald',
                        repo='Python_Tools',
                        folder='d_py_functions',
                        output_folder=""):
    '''
    Function to Download Files from Github to a dedicated folder. Specifically used when i DO NOT want to formally link to Github.
    
    Parameters:
        User:
        Repo:
        folder:
        output_folder:
        
    Returns:
        Saves files to Output Folder.
    

    '''
    
    if len(output_folder) == 0:
        output_folder = os.getcwd()
    
    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder}"
    response = requests.get(api_url)

    if response.status_code == 200:
        files = response.json()
        py_files = [file for file in files if file['name'].endswith('.py')]

        for file in py_files:
            file_url = file['download_url']
            file_name = file['name']
            file_response = requests.get(file_url)

            if file_response.status_code == 200:
                with open(os.path.join(output_folder, file_name), "w", encoding="utf-8") as f:
                    f.write(file_response.text)
                    
        return True
    else:
        return False

def ParamterMapping(Definition=""):
    
    '''
    Function to Google Mapping Sheet, which is used to store Mappings, Links, etc.
    For both simplicity and Organization
    
    Args:
        Definition (Str): Key word used to Access individual elements
        
    Returns:
        Dataframe, unless Definition is defined, in which case it might be Str.
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    # If user has not included a definition, the return entire DF
    if len(Definition)==0:
        return df
    else:
        try:
            df1 = df[df['Definition']==Definition]
            if len(df1)==1:
                if df1['TYPE'].item()=='csv':
                    return pd.read_csv(df1['VALUE'].item())
                else:
                    return df1['VALUE'].item()
        except:
            return df[df['Definition']==Definition] 


def BackUpGoogleSheets(location='/Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/'):
    '''
    Function to Create a Backup of Information Stored in Google Sheets.
    
    Parameters:
        None
        
    Returns:
        CSV Files 
    
    '''
    
    df = ParamterMapping()
    
    for row in range(len(df)):
        try:
            file_name = df['Definition'][row]
            file_location = df['CSV'][row]
            month = datetime.datetime.now().strftime('%b-%y')
            
            temp_df = pd.read_csv(file_location)
            temp_df.to_csv(f'{location}{file_name}_{month}.csv',index=False)
            print(f'Back Up Saved, {location}{file_name}')
        except:
            print(f'Counld Not Print Record {row}')





def NavigateUsingDMap():
     
    '''
    Function to Google Mapping Sheet, Navigate to Specific Sites.
    Provides Options, Enable Selection based on inputs.
    
    Parameters:
    
        
    Returns:
        
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    display(df)
    
    p = input('Which Process would You like to review?')
    v = input('What would you like to return?')
    
    df1 = df[df['Definition']==p]
    
    if v.lower() =='link':
        webbrowser.open(df1['Link'].item())
    elif v.lower() == 'csv':
        return pd.read_csv(df1['CSV'].item())
    elif v.lower()=='streamlit':
        webbrowser.open(df1['Streamlit'].item())
